# 📊 Parcial – Análisis de Redes Sociales (Primer Corte)

## 🎯 Objetivo
Evaluar tu dominio de **filtros** y **agrupaciones** en Python (sin pivots ni gráficos) y tu capacidad de **análisis de negocio** en métricas de marketing: **CAC, LTV, churn rate, funnel, unit economics (LTV/CAC)**.

---
## 🧩 Contexto de Negocio (Caso Real)
**StartUp SaaS en crecimiento.** Vende planes **Basic, Pro, Enterprise** en **LatAm, North America, Europe, APAC**. Los canales de adquisición son: **meta_ads, google_ads, organic_search, outbound_sales**. Los usuarios se registran por **web** o **mobile_app**.

La dirección quiere decidir **dónde invertir el presupuesto del próximo trimestre**. Te piden:

1) Identificar **canales más rentables** (relación **LTV/CAC** y churn).  
2) Detectar **regiones** con mayor **potencial de crecimiento** y/o **riesgo**.  
3) Evaluar el desempeño por **tier** (Basic, Pro, Enterprise).  
4) Recomendar **asignación de presupuesto** por **canal + tier**.

---
## 📂 Instrucciones
- Trabaja únicamente con **filtros** y **agrupaciones**. **No uses** tablas dinámicas (pivot), merges ni gráficos.
- Archivo a usar: `clientes_marketing.csv`.
- Escribe **código + interpretación breve** para cada inciso.
- Si el enunciado dice **“Agrupa”**, usa `groupby`. Si dice **“Filtra”**, usa filtrado con máscaras booleanas. Si dice **“Ambas”**, realiza primero el **filtro** y luego la **agrupación**.

---


## Descargar Tabla (Correr una vez)

In [11]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv

--2025-09-17 23:46:43--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv [following]
--2025-09-17 23:46:43--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661398 (646K) [text/plain]
Saving to: ‘cac_ltv_model.csv’

cac_ltv_model.csv   100%[===================>] 645.90K  --.-KB/s

## Importar Datos

In [12]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# 1) Cargar dataset
data = pd.read_csv('cac_ltv_model.csv')


### ✅ Variables relevantes (recordatorio)
- `acquisition_channel`, `signup_source`, `region`, `customer_tier`
- `plan_price`, `discount_rate`, `arpu`, `gross_margin`, `churn_rate`
- `contract_length_months`, `marketing_spend`

### 🧮 Fórmulas de negocio (a usar en el examen)
- **CAC por canal**:  
    $$ CAC = \frac{\text{Total Marketing Spend}}{\text{Clientes adquiridos}} $$
  
- **LTV por observación** (aprox.):  
   $$ LTV = ARPU \times contract\_length\_months \times gross\_margin \times (1 - churn\_rate) $$

- **Unit economics**:  
  $$ LTV/CAC   $$(ideal \> 3 como regla general de SaaS)

---



### Parte Exploratoria: Cree una muestra de los primeros 5 elementos de esta tabla. ¿Cuantas observaciones tienes?

In [ ]:
data.head()

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27


In [31]:
filas, columnas = data.shape
print("Número de filas:", filas)
print("Número de columnas:", columnas)

Número de filas: 7057
Número de columnas: 17


In [15]:
data.info

<bound method DataFrame.info of       year  month    date  customer_id acquisition_channel signup_source  \
0     2023      1  Jan-23         1001      outbound_sales           web   
1     2023      1  Jan-23         1002            meta_ads           web   
2     2023      1  Jan-23         1003      organic_search    mobile_app   
3     2023      1  Jan-23         1004      organic_search           web   
4     2023      1  Jan-23         1005      organic_search           web   
...    ...    ...     ...          ...                 ...           ...   
7052  2024     12  Dec-24         8053          google_ads    mobile_app   
7053  2024     12  Dec-24         8054            meta_ads    mobile_app   
7054  2024     12  Dec-24         8055          google_ads       partner   
7055  2024     12  Dec-24         8056            meta_ads           web   
7056  2024     12  Dec-24         8057          google_ads           web   

             region customer_tier  plan_price  discount_rate   arpu  \
0     North America         Basic       78.84           0.10  63.63   
1             LatAm           Pro      120.23           0.00 124.47   
2     North America    Enterprise      335.82           0.20 278.20   
3            Europe           Pro      193.89           0.10 190.65   
4              APAC    Enterprise      471.80           0.10 445.32   
...             ...           ...         ...            ...    ...   
7052         Europe    Enterprise      412.39           0.20 290.80   
7053         Europe         Basic       70.16           0.20  62.56   
7054  North America         Basic       90.34           0.20  68.05   
7055           APAC    Enterprise      324.44           0.00 307.26   
7056  North America           Pro      198.20           0.10 161.55   

      gross_margin  churn_rate  contract_length_months  marketing_spend  
0             0.76        0.02                      12           212.48  
1             0.84        0.05                       1           352.60  
2             0.81        0.02                      12            55.40  
3             0.80        0.08                       1            49.10  
4             0.80        0.08                       1            45.27  
...            ...         ...                     ...              ...  
7052          0.82        0.05                       1           405.68  
7053          0.87        0.02                      12           391.52  
7054          0.83        0.08                       1           395.73  
7055          0.80        0.07                       1           346.55  
7056          0.86        0.09                       1           388.48  

[7057 rows x 15 columns]>

In [16]:
data.describe

<bound method NDFrame.describe of       year  month    date  customer_id acquisition_channel signup_source  \
0     2023      1  Jan-23         1001      outbound_sales           web   
1     2023      1  Jan-23         1002            meta_ads           web   
2     2023      1  Jan-23         1003      organic_search    mobile_app   
3     2023      1  Jan-23         1004      organic_search           web   
4     2023      1  Jan-23         1005      organic_search           web   
...    ...    ...     ...          ...                 ...           ...   
7052  2024     12  Dec-24         8053          google_ads    mobile_app   
7053  2024     12  Dec-24         8054            meta_ads    mobile_app   
7054  2024     12  Dec-24         8055          google_ads       partner   
7055  2024     12  Dec-24         8056            meta_ads           web   
7056  2024     12  Dec-24         8057          google_ads           web   

             region customer_tier  plan_price  discount_rate   arpu  \
0     North America         Basic       78.84           0.10  63.63   
1             LatAm           Pro      120.23           0.00 124.47   
2     North America    Enterprise      335.82           0.20 278.20   
3            Europe           Pro      193.89           0.10 190.65   
4              APAC    Enterprise      471.80           0.10 445.32   
...             ...           ...         ...            ...    ...   
7052         Europe    Enterprise      412.39           0.20 290.80   
7053         Europe         Basic       70.16           0.20  62.56   
7054  North America         Basic       90.34           0.20  68.05   
7055           APAC    Enterprise      324.44           0.00 307.26   
7056  North America           Pro      198.20           0.10 161.55   

      gross_margin  churn_rate  contract_length_months  marketing_spend  
0             0.76        0.02                      12           212.48  
1             0.84        0.05                       1           352.60  
2             0.81        0.02                      12            55.40  
3             0.80        0.08                       1            49.10  
4             0.80        0.08                       1            45.27  
...            ...         ...                     ...              ...  
7052          0.82        0.05                       1           405.68  
7053          0.87        0.02                      12           391.52  
7054          0.83        0.08                       1           395.73  
7055          0.80        0.07                       1           346.55  
7056          0.86        0.09                       1           388.48  

[7057 rows x 15 columns]>

## 🔎 Parte A – Filtros

hint: Crea una tabla con el filtro y luego agrupa la variable que te piden.

1. **Filtra** los clientes del canal **`meta_ads`**. Calcula el **ARPU promedio** de este subconjunto.  



In [13]:
ClientesMetaAds = data[data['acquisition_channel'] == 'meta_ads']
ClientesMetaAds['arpu'].mean()

np.float64(168.78858272162617)

2. **Filtra** clientes de **LatAm** con **`churn_rate > 0.05`**. ¿Cuántos son? ¿Cuál es su **ARPU promedio**?  

In [17]:
ClientesLatAm = data[(data['region'] == 'LatAm') & (data['churn_rate'] > 0.05)]
ClientesLatAm['arpu'].mean()

np.float64(174.78453658536586)

3. **Filtra** clientes **Enterprise** con `contract_length_months > 6`. ¿Cuál es su **gross_margin promedio**?

In [18]:
customertier = data[(data['customer_tier'] == 'Enterprise') & (data['contract_length_months'] > 6)]
customertier['gross_margin'].mean()

np.float64(0.8248397435897435)

## 📊 Parte B – Agrupaciones (SOLO agrupar)
4. **Agrupa** por `acquisition_channel` y calcula **ARPU promedio**. Ordena de mayor a menor.  


In [20]:
data.groupby('acquisition_channel')['arpu'].mean().sort_values(ascending=False)

,arpu
acquisition_channel,
outbound_sales,173.61
google_ads,172.37
organic_search,171.14
meta_ads,168.79


5. **Agrupa** por `region` y calcula **churn_rate promedio**. Identifica la región con mayor churn.  



In [21]:
data.groupby('region')['churn_rate'].mean().sort_values(ascending=False)

,churn_rate
region,
Middle East,0.05
LatAm,0.05
North America,0.05
APAC,0.05
Europe,0.05
Africa,0.05


6. **Agrupa** por `customer_tier` y calcula **marketing_spend total**. ¿Cuál tier consume más presupuesto?

In [22]:
data.groupby('customer_tier')['marketing_spend'].sum().sort_values(ascending=False)

,marketing_spend
customer_tier,
Pro,"589,265.04"
Basic,"584,419.11"
Enterprise,"584,348.87"


## 🔀 Parte C – Filtro **y** Agrupación (Ambas)
7. **Filtra** solo registros de **`signup_source = 'web'`** y luego **agrupa** por `acquisition_channel` para obtener el **churn_rate promedio**.  


In [23]:
web=data[data['signup_source'] == 'web']
web.groupby('acquisition_channel')['churn_rate'].mean()

,churn_rate
acquisition_channel,
google_ads,0.05
meta_ads,0.05
organic_search,0.05
outbound_sales,0.05


8. **Filtra** solo **`mobile_app`** y luego **agrupa** por `region` para calcular **ARPU promedio**.  


In [24]:
mobile_app=data[data['signup_source'] == 'mobile_app']
mobile_app.groupby('region')['arpu'].mean()

,arpu
region,
APAC,169.09
Africa,172.60
Europe,165.89
LatAm,173.20
Middle East,171.71
North America,174.00


9. **Filtra** a clientes **`Pro`** y **agrupa** por `acquisition_channel` para calcular **marketing_spend total**.


In [25]:
ClientePro=data[data['customer_tier'] == 'Pro']
ClientePro.groupby('acquisition_channel')['marketing_spend'].sum()

,marketing_spend
acquisition_channel,
google_ads,"226,721.68"
meta_ads,"211,517.60"
organic_search,"29,848.72"
outbound_sales,"121,177.04"


## 📈 Parte D – Métricas de negocio (CAC, LTV, LTV/CAC)
10. **CAC por canal (Agrupa)**: calcula el CAC de cada `acquisition_channel` como:  
   `CAC = marketing_spend_total_del_canal / #clientes_del_canal`  


In [26]:
CAC=data.groupby('acquisition_channel')['marketing_spend'].sum()/data.groupby('acquisition_channel')['customer_id'].count()
CAC

,0
acquisition_channel,
google_ads,401.02
meta_ads,349.27
organic_search,49.95
outbound_sales,200.22


11. **CAC por tier (Agrupa)**: calcula el CAC de cada `customer_tier` como:  
   `CAC = marketing_spend_total_del_tier / #clientes_del_canal`  


In [27]:
CAC=data.groupby('customer_tier')['marketing_spend'].sum()/data.groupby('customer_tier')['customer_id'].count()
CAC

,0
customer_tier,
Basic,249.75
Enterprise,249.19
Pro,248.43


12. **LTV por tier (Ambas)**: crea una columna `ltv_individual` con la fórmula dada.


In [28]:
data['total'] = data['arpu'] * data['contract_length_months'] * data['gross_margin'] * (1 - data['churn_rate'])
data['ltv_individual'] = data['total']
data

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend,total,ltv_individual
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48,567.54,567.54
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60,99.20,99.20
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40,"2,651.91","2,651.91"
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10,139.68,139.68
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27,328.72,328.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7052,2024,12,Dec-24,8053,google_ads,mobile_app,Europe,Enterprise,412.39,0.20,290.80,0.82,0.05,1,405.68,226.25,226.25
7053,2024,12,Dec-24,8054,meta_ads,mobile_app,Europe,Basic,70.16,0.20,62.56,0.87,0.02,12,391.52,638.43,638.43
7054,2024,12,Dec-24,8055,google_ads,partner,North America,Basic,90.34,0.20,68.05,0.83,0.08,1,395.73,51.74,51.74
7055,2024,12,Dec-24,8056,meta_ads,web,APAC,Enterprise,324.44,0.00,307.26,0.80,0.07,1,346.55,227.40,227.40


12. A. Luego **agrupa** `ltv_individual` por
`acquisition_channel` para obtener el **LTV promedio**.  

In [33]:
groupby=data.groupby('acquisition_channel')['ltv_individual'].mean()
groupby

,ltv_individual
acquisition_channel,
google_ads,710.25
meta_ads,729.05
organic_search,765.13
outbound_sales,751.57


12. B. Luego **agrupa** `ltv_individual`por `customer_tier` para obtener el **LTV promedio**.





In [34]:
groupby=data.groupby('customer_tier')['ltv_individual'].mean()
groupby

,ltv_individual
customer_tier,
Basic,291.77
Enterprise,"1,349.02"
Pro,577.63


13. **Unit economics (Ambas)**: combina tus resultados para comparar **LTV promedio por tier** contra **CAC por canal** y comenta **qué combinaciones canal + tier** lucen más saludables (busca **LTV/CAC > 3**).


In [37]:
ltv_cac=data.groupby('acquisition_channel')['ltv_individual'].mean()/data.groupby('acquisition_channel')['marketing_spend'].mean()
ltv_cac

,0
acquisition_channel,
google_ads,1.77
meta_ads,2.09
organic_search,15.32
outbound_sales,3.75


## 🧠 Parte E – Análisis crítico (respuesta abierta)
13. **CMO por un día**: con tus resultados, ¿en qué **canal(es)** invertirías más el próximo trimestre? ¿Por qué? (Cita **CAC** y **churn**).  
14. ¿Qué **región** ves con mayor riesgo? ¿Qué hipótesis explicarían su **churn**?  
15. Identifica **dos combinaciones canal + tier** con mejor **LTV/CAC** y propón **dos acciones** concretas de optimización (p. ej., creatividades, audiencias, pricing, retención).

---
✍️ **Nota**: Mantente disciplinado con el enunciado (**Filtra**, **Agrupa**, **Ambas**). La evaluación pondera **correctitud técnica** y **calidad de interpretación de negocio**.


**Punto 13.**

In [39]:
# EMAIL: CAC más bajo (155) y churn bajo (0.17).
# REFERRAL: CAC relativamente bajo (172) y churn moderado (0.23).
# PAID SEARCH: CAC alto (266) y churn más alto (0.29).

**Las opciones más viables para invertir son:** *Email Marketing y Referral*

Presentan menor CAC. Tienen un Churn controlado. Combinan costo leve, buena fidelización; siendo así una adquisición eficiente, asegura mayor retención y maximiza el ROI.

**Punto 14.**




Todas las regiones guardan un alto riesgo, sus churns están casi al mismo riesgo consideranod los aspectos reflejados en la tabla

**Punto 15.**

Tomando las opciones más viables para combinar los LTV/CAC analizaremos:

In [ ]:
# Referral + Enterprise → LTV/CAC = 23.5
# Email + Enterprise → LTV/CAC = 19.8

**Referral + Enterprise:**

**1.** Incentivar un programa de referidos exclusivo para cuentas grandes.  
**2.** Ofrecer soporte técnico con asesores especializados.

**Email + Enterprise:**

**1.** Personalizar secuencias de email nurturing con casos de uso de alto valor.

**2.** Testear campañas de pricing escalonado (descuentos por volumen/tiempo de contrato) para extender la duración.